In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install praw

In [3]:
import praw
import json

reddit = praw.Reddit(
    client_id="5bX6wjvW6jVAPUs4NYTWZg",
    client_secret="kvcIOCVIU5N-Rv_kfR1Rs5zmGUwxgQ",
    user_agent="amg-chatbot-user-agentr"
)

def fetch_reddit_comments(subreddit, lang, limit=500):
    posts = []
    for comment in reddit.subreddit(subreddit).comments(limit=limit):
        text = comment.body.strip()
        if len(text) > 20:
            posts.append({
                "text": text,
                "language": lang,
                "source": f"reddit:{subreddit}",
                "sentiment": None
            })
    return posts

# Example: English + German
en_posts = fetch_reddit_comments("technology", "en", 500)
de_posts = fetch_reddit_comments("de", "de", 500)

with open("reddit_praw_posts.jsonl", "w", encoding="utf-8") as f:
    for post in en_posts + de_posts:
        f.write(json.dumps(post, ensure_ascii=False) + "\n")

print(f"Saved {len(en_posts) + len(de_posts)} Reddit posts!")


Saved 961 Reddit posts!


In [4]:
from transformers import pipeline
import json

# Load your Reddit posts
def load_posts(file_path):
    posts = []
    with open(file_path, encoding="utf-8") as f:
        for line in f:
            posts.append(json.loads(line.strip()))
    return posts

# Auto-label sentiment
def label_sentiment(posts, limit=2000):
    sentiment_analyzer = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
    labeled = []
    for post in posts[:limit]:
        result = sentiment_analyzer(post["text"][:512])[0]  # Truncate long texts
        stars = int(result["label"].split()[0])  # e.g. "4 stars" -> 4
        if stars >= 4:
            sent = "positive"
        elif stars == 3:
            sent = "neutral"
        else:
            sent = "negative"
        post["sentiment"] = sent
        labeled.append(post)
    return labeled

# Save labeled data
def save_labeled(posts, out_file):
    with open(out_file, "w", encoding="utf-8") as f:
        for post in posts:
            f.write(json.dumps(post, ensure_ascii=False) + "\n")
    print(f"✅ Saved {len(posts)} labeled posts to {out_file}")

# Example
reddit_posts = load_posts("reddit_praw_posts.jsonl")
labeled_posts = label_sentiment(reddit_posts, limit=2000)
save_labeled(labeled_posts, "reddit_labeled_posts.jsonl")


2025-07-07 01:45:04.740111: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751852704.763785     170 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751852704.770663     170 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


✅ Saved 961 labeled posts to reddit_labeled_posts.jsonl


In [10]:
import json

def load_yt_comments(file_path):
    with open(file_path, encoding="utf-8") as f:
        data = json.load(f)

    posts = []
    for c in data.get("comments", []):
        text = c.get("text", "").strip()
        if len(text) > 15:  # skip super short stuff
            posts.append({
                "text": text,
                "language": "en",  # adjust if you know comments are German
                "source": "youtube",
                "sentiment": None
            })
    return posts

yt_posts = load_yt_comments("New AMG GT review： Better than a 911？! [2vWh2kaeZkI].info.json")
print(f"✅ Loaded {len(yt_posts)} YouTube comments")



✅ Loaded 2761 YouTube comments


In [11]:
import json

# Load Reddit posts
with open("reddit_praw_posts.jsonl", encoding="utf-8") as f:
    reddit_posts = [json.loads(line.strip()) for line in f]

# Now combine
combined_posts = reddit_posts + yt_posts
print(f"🧮 Combined total posts: {len(combined_posts)}")

# Save to file
with open("all_social_posts.jsonl", "w", encoding="utf-8") as f:
    for post in combined_posts:
        f.write(json.dumps(post, ensure_ascii=False) + "\n")

print("✅ Saved combined dataset as all_social_posts.jsonl")


🧮 Combined total posts: 3722
✅ Saved combined dataset as all_social_posts.jsonl


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import json

# Load model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

def get_sentiment(text):
    encoded = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    output = model(**encoded)
    scores = softmax(output.logits.detach().numpy()[0])
    labels = ["negative", "neutral", "positive"]
    return labels[scores.argmax()]


# Load combined data
with open("all_social_posts.jsonl", encoding="utf-8") as f:
    posts = [json.loads(line.strip()) for line in f]

# Label ~2,000 posts
labeled_posts = []
for post in posts[:2000]:
    text = post.get("text", "")
    sentiment = get_sentiment(text)
    post["sentiment"] = sentiment
    labeled_posts.append(post)

# Save
with open("labeled_social_posts.jsonl", "w", encoding="utf-8") as f:
    for post in labeled_posts:
        f.write(json.dumps(post, ensure_ascii=False) + "\n")

print(f"✅ Labeled and saved {len(labeled_posts)} posts")


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

✅ Labeled and saved 2000 posts


In [13]:
# Install sentence-transformers if not already done
!pip install -U sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
import json

# Load model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 58.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully unins

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
import json
import pickle
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Load posts
with open("all_social_posts.jsonl", encoding="utf-8") as f:
    posts = [json.loads(line.strip()) for line in f]

# Extract + clean
texts = [post.get("text", "") for post in posts]
clean_texts = [t.strip() for t in texts if len(t.strip()) > 30]
print(f"✅ Filtered down to {len(clean_texts)} clean posts.")

# Embed
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = model.encode(clean_texts, batch_size=64, show_progress_bar=True).astype("float32")

# Build index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# Save
faiss.write_index(index, "rag_index_clean.faiss")
with open("rag_texts_clean.pkl", "wb") as f:
    pickle.dump(clean_texts, f)

print("✅ Clean FAISS index and texts saved.")


✅ Filtered down to 3190 clean posts.


Batches:   0%|          | 0/50 [00:00<?, ?it/s]

✅ Clean FAISS index and texts saved.


In [15]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load generator
gen_model_name = "google/flan-t5-large"  # You can change to flan-t5-large or another
tokenizer_gen = AutoTokenizer.from_pretrained(gen_model_name)
model_gen = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

# Load index + texts
index = faiss.read_index("rag_index_clean.faiss")
with open("rag_texts_clean.pkl", "rb") as f:
    clean_texts = pickle.load(f)

# Function: retrieve
def retrieve_top_k(query, embed_model, index, texts, k=5):
    query_vec = embed_model.encode([query]).astype("float32")
    D, I = index.search(query_vec, k)
    return [texts[i] for i in I[0]]

# Function: generate
def generate_answer(query, retrieved, tokenizer, model, max_tokens=128):
    context = "\n".join(retrieved)
    prompt = (
        f"Here are some customer comments:\n"
        f"{context}\n"
        f"Based on these, provide a clear, helpful, natural-language answer to the question:\n"
        f"{query}"
    )
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    output = model.generate(**inputs, max_length=max_tokens)
    return tokenizer.decode(output[0], skip_special_tokens=True), retrieved

# Example usage
query = "What do customers think about the AMG GT XX?"
retrieved = retrieve_top_k(query, model, index, clean_texts, k=5)
answer, refs = generate_answer(query, retrieved, tokenizer_gen, model_gen)

print(f"🔹 Answer: {answer}\n")
print("🔹 Retrieved snippets:")
for r in refs:
    print("-", r[:300])


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🔹 Answer: It's an SL with AMG branding and nothing more.

🔹 Retrieved snippets:
- Imagine thinking this is prettier than the original 2016 AMG GT.
- That new AMG GT looks amazing! 😎👍🏾. I’m looking forward to seeing it in a drag race. 😎
- I can alleviate a lot of the confusion in the comments section. This is not a true AMG GT. It’s an SL with AMG branding and nothing more. These car reviewers have to maintain good relationships with the manufacturers so you’ll hear them skim over that like Matt did here.

Real AMG GT: SLS based, RWD
- Thanks for the review, encouraged me to make the decision to shortlist the new AMG GT63. I'm gonna take it for a test drive next time I go to sleep and have a dream.
- nobody really buys an amg gt for daily driving , with that said i dont think this is an improvement on the last gen , it doesnt look necessarily better , its not more powerfull , its 400kg heavier and the interior isnt that much better either . I think the older generation car will apreciat

In [16]:
import json
import pickle
import faiss
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# --- Load models ---
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
gen_model_name = "google/flan-t5-base"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

# --- Load FAISS + texts ---
index = faiss.read_index("rag_index_clean.faiss")
with open("rag_texts_clean.pkl", "rb") as f:
    texts = pickle.load(f)

# --- Topics + aspects ---
topics = [
    # AMG models
    "AMG GT XX", "AMG GT", "AMG GT Black Series", "AMG C63", "AMG E63", "AMG A45",
    "AMG GLC 63", "AMG GLE 63", "AMG S63", "AMG CLA 45", "AMG SL63",
    # BMW
    "BMW M3", "BMW M5", "BMW X5M", "BMW i4", "BMW iX", "BMW X3", "BMW M2",
    # Audi
    "Audi RS6", "Audi RS7", "Audi S3", "Audi Q8", "Audi e-tron", "Audi RS Q8",
    # Porsche
    "Porsche 911", "Porsche Taycan", "Porsche Macan", "Porsche Cayenne", "Porsche Panamera"
]

aspects = [
    "What do customers think about", "What do people dislike about",
    "What do customers like about", "Is it worth the money?",
    "What do people say about the design of", "What are common complaints about",
    "What positive feedback is there for", "What negative feedback is there for",
    "How do people feel about the price of", "How reliable is",
    "How is the comfort of", "What about the safety features of",
    "Is the tech in", "What about fuel efficiency of",
    "Is it fun to drive", "How does the interior of",
    "What about the build quality of", "How does the handling feel for",
    "How practical is", "What about resale value of"
]

# --- Build queries ---
queries = [f"{aspect} {topic}?" for topic in topics for aspect in aspects]

# Add comparison queries
compare_templates = [
    "How does {} compare to {}?",
    "Which is better: {} or {}?",
    "Why choose {} over {}?"
]

for i, a in enumerate(topics):
    for b in topics[i+1:]:
        for tmpl in compare_templates:
            queries.append(tmpl.format(a, b))

print(f"✅ Total queries to generate: {len(queries)}")

# --- Generate Q+A pairs ---
output_file = "qa_finetune_data_instructlab2.jsonl"
with open(output_file, "w", encoding="utf-8") as f_out:
    for query in tqdm(queries, desc="Generating Q+A pairs"):
        try:
            query_vec = embed_model.encode([query]).astype("float32")
            D, I = index.search(query_vec, k=5)
            retrieved = [texts[i] for i in I[0]]
            context = " ".join(retrieved)

            prompt = f"Question: {query}\nContext: {context}\nAnswer:"
            inputs = gen_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
            outputs = gen_model.generate(**inputs, max_length=100)
            answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

            record = {"input": query, "target": answer}
            f_out.write(json.dumps(record, ensure_ascii=False) + "\n")
        except Exception as e:
            print(f"⚠️ Skipped query due to error: {query}\nError: {e}")

print(f"✅ Saved: {output_file}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Q+A pairs: 100%|█████████▉| 1797/1798 [28:43<00:01,  1.11s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Q+A pairs: 100%|██████████| 1798/1798 [28:45<00:00,  1.04it/s]

✅ Saved: qa_finetune_data_instructlab2.jsonl


In [17]:
import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments

# 1️⃣ Load Q+A data
data = []
with open("qa_finetune_data_instructlab2.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))
print(f"Loaded {len(data)} Q+A pairs.")

dataset = Dataset.from_list(data)

# 2️⃣ Load model + tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 3️⃣ Preprocess
def preprocess(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples["target"], padding="max_length", truncation=True, max_length=100)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=dataset.column_names)

# 4️⃣ Training config
training_args = Seq2SeqTrainingArguments(
    output_dir="./flan_t5_finetuned_output1",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    logging_steps=100,
    report_to=[],
    remove_unused_columns=False
)

# 5️⃣ Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

# 6️⃣ Train
trainer.train()


Loaded 1798 Q+A pairs.


Map:   0%|          | 0/1798 [00:00<?, ? examples/s]

/tmp/ipykernel_170/3379187564.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,10.295000
200,0.758700
300,0.359600
400,0.289000
500,0.247600
600,0.274500
700,0.230300
800,0.226000
900,0.198400
1000,0.191400


TrainOutput(global_step=1350, training_loss=1.0197178113019025, metrics={'train_runtime': 539.5507, 'train_samples_per_second': 9.997, 'train_steps_per_second': 2.502, 'total_flos': 3693581159104512.0, 'train_loss': 1.0197178113019025, 'epoch': 3.0})

In [18]:
!zip -r flan_t5_finetuned_output1.zip flan_t5_finetuned_output1

  adding: flan_t5_finetuned_output1/ (stored 0%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/ (stored 0%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/spiece.model (deflated 48%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/special_tokens_map.json (deflated 85%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/trainer_state.json (deflated 71%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/tokenizer.json (deflated 74%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/tokenizer_config.json (deflated 95%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/optimizer.pt (deflated 17%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/rng_state.pth (deflated 25%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/generation_config.json (deflated 29%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/scheduler.pt (deflated 55%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/config.json (deflated 62%)
  adding: flan_t5_finetuned_output1/checkpoint-1000/training_args.bin (deflated 51%)
  adding: flan_t5_finetuned_output1/checkpoint-1350/ (stored 0%)
  adding: flan_t5_finetuned_output1/checkpoint-1350/spiece.model (deflated 48%)
  adding: flan_t5_finetuned_output1/checkpoint-1350/special_tokens_map.json (deflated 85